In [28]:
"""
see: https://github.com/BLarzalere/LSTM-Autoencoder-for-Anomaly-Detection/blob/master/Sensor%20Anomaly%20Detection.ipynb
"""
import pandas as pd
from tqdm import tqdm

In [31]:
# Create a bucket in the current project
BUCKET_NAME = "dlaw_bucket"
!gsutil mb gs://$BUCKET_NAME

Creating gs://dlaw_bucket/...
ServiceException: 409 Bucket dlaw_bucket already exists.


In [40]:
source_bucket_name = "amazing-public-data"
merged_data = pd.DataFrame()

client = storage.Client()
prefix = "bearing_sensor_data/bearing_sensor_data/"
blobs = client.list_blobs(source_bucket_name, prefix=prefix)

files = []
for blob in tqdm(blobs, total=982):
    dataset = pd.read_csv("gs://{0}/{1}".format(source_bucket_name, blob.name), sep='\t')
    dataset_mean_abs = np.array(dataset.abs().mean())
    dataset_mean_abs = pd.DataFrame(dataset_mean_abs.reshape(1, 4))
    dataset_mean_abs.index = [blob.name.split("/")[-1]]
    merged_data = merged_data.append(dataset_mean_abs)
    files.append(blob.name)

merged_data.columns = ['bearing-1', 'bearing-2', 'bearing-3', 'bearing-4']
print(len(files))

100%|██████████| 982/982 [01:45<00:00,  9.32it/s]

982


In [41]:
merged_data.head()

,bearing-1,bearing-2,bearing-3,bearing-4
2004.02.12.10.52.39,0.060236,0.074227,0.083926,0.044443
2004.02.12.11.02.39,0.061455,0.073844,0.084457,0.045081
2004.02.12.11.12.39,0.061361,0.075609,0.082837,0.045118
2004.02.12.11.22.39,0.061665,0.073279,0.084879,0.044172
2004.02.12.11.32.39,0.061944,0.074593,0.082626,0.044659


In [42]:
# transform data file index to datetime and sort in chronological order
merged_data.index = pd.to_datetime(merged_data.index, format='%Y.%m.%d.%H.%M.%S')
merged_data = merged_data.sort_index()
print("Dataset shape:", merged_data.shape)
merged_data.head()

Dataset shape: (982, 4)


,bearing-1,bearing-2,bearing-3,bearing-4
2004-02-12 10:52:39,0.060236,0.074227,0.083926,0.044443
2004-02-12 11:02:39,0.061455,0.073844,0.084457,0.045081
2004-02-12 11:12:39,0.061361,0.075609,0.082837,0.045118
2004-02-12 11:22:39,0.061665,0.073279,0.084879,0.044172
2004-02-12 11:32:39,0.061944,0.074593,0.082626,0.044659


In [43]:
DEST_FILE_NAME = "raw_bearing_data.csv"
merged_data.to_csv(f"gs://{BUCKET_NAME}/{DEST_FILE_NAME}")